In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data

(train_x, train_y), (test_x, test_y) = load_data()
train_x.shape

2024-10-31 11:29:22.983388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730370562.999423   69944 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730370563.003965   69944 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 11:29:23.019701: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


(60000, 28, 28)

In [3]:
train_x = train_x / 255.0
test_x = test_x / 255.0

train_x = train_x[..., tf.newaxis].astype("float32")
test_x = test_x[..., tf.newaxis].astype("float32")

In [4]:
from tensorflow.keras.models import Sequential

model = Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  # tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/home/lars/.venv/tensorflow/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1730370565.125633   69944 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7324 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


In [5]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_x, train_y,epochs=5,batch_size=32)

Epoch 1/5


I0000 00:00:1730370566.930187   70216 service.cc:148] XLA service 0x7a56a4007da0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730370566.930211   70216 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-10-31 11:29:26.944335: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1730370566.992832   70216 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-10-31 11:29:27.272890: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12.5.82. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


  97/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5640 - loss: 1.4175

I0000 00:00:1730370567.516913   70216 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8707 - loss: 0.4438
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9656 - loss: 0.1223
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9757 - loss: 0.0808
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9808 - loss: 0.0603
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9864 - loss: 0.0444


In [6]:
scores = model.evaluate(test_x, test_y)
scores

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9731 - loss: 0.0900


[0.07734528928995132, 0.9775000214576721]

In [7]:
from scikeras.wrappers import KerasClassifier

from sklearn.model_selection import GridSearchCV

def create(drop_out):
    model = Sequential([
          tf.keras.layers.Flatten(input_shape=(28, 28)),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dropout(drop_out),
          tf.keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

classifier = KerasClassifier(create, drop_out=0.1)

params = {'drop_out':[0.1, 0.2, 0.3]}
grid = GridSearchCV(classifier, params)
grid.fit(train_x, train_y)

/home/lars/.venv/tensorflow/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8538 - loss: 0.5041
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


/home/lars/.venv/tensorflow/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8563 - loss: 0.4939
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


/home/lars/.venv/tensorflow/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/home/lars/.venv/tensorflow/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8333 - loss: 0.5603
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


/home/lars/.venv/tensorflow/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8700 - loss: 0.4542


GridSearchCV(estimator=KerasClassifier(drop_out=0.1, model=<function create at 0x7a577953ae80>),
             param_grid={'drop_out': [0.1, 0.2, 0.3]})

In [11]:
results = pd.DataFrame(grid.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_drop_out,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.188768,0.171483,0.844886,0.047409,0.1,{'drop_out': 0.1},0.950417,0.950250,0.952833,0.946500,0.956083,0.951217,0.003166,1
1,4.363728,0.111141,0.869656,0.008331,0.2,{'drop_out': 0.2},0.948667,0.949250,0.948583,0.945417,0.955750,0.949533,0.003386,2
2,4.397132,0.032812,0.920736,0.120106,0.3,{'drop_out': 0.3},0.949000,0.948917,0.946000,0.946000,0.953917,0.948767,0.002895,3


In [16]:
grid.best_estimator_.model_.evaluate(test_x, test_y)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9507 - loss: 0.1554


[0.13653400540351868, 0.9574999809265137]

In [10]:
from tensorflow.keras.layers import Softmax

final_model = Sequential([model, Softmax()])
final_model(test_x[:1])

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[1.0418396e-07, 1.8711390e-08, 2.2321467e-06, 3.7750517e-05,
        2.7379325e-11, 2.4070850e-08, 5.6403813e-13, 9.9995303e-01,
        1.0168037e-06, 5.8958440e-06]], dtype=float32)>